In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')


import re
import os, sys

import numpy as np # linear algebra

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

from config import SEED, PARTIAL_TRAIN, TEST_SIZE, NUM_LABELS 
from config import MAX_SEQUENCE_LENGTH, NUM_EPOCH, LEARNING_RATE, BATCH_SIZE
from config import ACCUMULATION_STEPS, INPUT_DIR, WORK_DIR, TOXICITY_COLUMN, DATA_DIR
from config import BERT_MODEL_NAME, FINE_TUNED_MODEL_PATH

from utils import set_seed, convert_lines_onfly, preprocess

device = torch.device('cuda')

In [2]:
set_seed(SEED)

In [3]:
## instantiate bert pretrained model and tokenizer
model = BertForSequenceClassification.from_pretrained(BERT_MODEL_NAME, num_labels=NUM_LABELS)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

## load saved model
model.load_state_dict(torch.load(FINE_TUNED_MODEL_PATH)) 
for p in model.parameters():
    p.requires_grad = False

## use gpu
model.cuda()

def predict_toxicity(sentence: str) -> float :
    """
    predict the toxicity level from a sentence
    """    

    model.eval()
    
    X = np.array([str(sentence)])
    test_preds = torch.zeros((len(X)))
    
    Xp = convert_lines_onfly(X, MAX_SEQUENCE_LENGTH, tokenizer)
    y_pred = model(torch.from_numpy(Xp).to(device)).logits
    test_preds[0] = test_preds[0] + torch.sigmoid(y_pred[0,0].cpu())

    return float(test_preds[0])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
sentence = "Oh sh*t!! What an awesome goal, I nearly missed it…"
# sentence = "Yet call out all Muslims for the acts of a few will get you pilloried.   So why is it okay to smear an entire religion over these few idiots?  Or is this because it's okay to bash Christian sects?"
# sentence = "Sorry to have to do this, but just to see if profanity filtering is enabled"

predict_toxicity(sentence)

0.673354983329773